# Data ingestion

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

In this notebook we download the images corresponding to the slide deck that we uploaded into Amazon S3 in the [1_data_prep.ipynb](./1_data_prep) notebook, convert them into embeddings and then ingest these embeddings into a vector database i.e. [Amazon OpenSearch Service Serverless](https://aws.amazon.com/opensearch-service/features/serverless/).

1. We use the [Amazon Titan Multiodal Embeddings](https://aws.amazon.com/about-aws/whats-new/2023/11/amazon-titan-multimodal-embeddings-model-bedrock/) model to convert the images into embeddings.

1. The embeddings are then ingested into OpenSearch Service Serverless using the [Amazon OpenSearch Ingestion](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/ingestion.html) pipeline. The embeddings are uploaded into an S3 bucket and that triggers the OpenSearch Ingestion pipeline which ingests the data into an OpenSearch Serverless index.

1. The OpenSearch Service Serverless Collection is created via the AWS CloudFormation stack for this blog post.


## Step 1. Setup

Install the required Python packages and import the relevant files.

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
import os
import glob
import json
import time
import boto3
import codecs
import base64
import logging
import botocore
import numpy as np
import globals as g
from typing import List
from pathlib import Path
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from utils import upload_to_s3, get_cfn_outputs, get_bucket_name, download_image_files_from_s3

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name: str = get_bucket_name(g.CFN_STACK_NAME)

## Step 2. Download the images files from S3 and convert to Base64 

Now we download the image files from the S3 bucket. Once downloaded these files are converted into [Base64](https://en.wikipedia.org/wiki/Base64) encoding so that we can create embeddings from the images.

In [4]:
# download images from S3, we would be converting these to embeddings
image_files: List = download_image_files_from_s3(bucket_name, g.BUCKET_IMG_PREFIX, g.IMAGE_DIR, g.IMAGE_FILE_EXTN)
logger.info(f"downloaded {len(image_files)} from s3")

[2024-01-25 22:17:16,113] p16253 {utils.py:33} INFO - downloaded multimodal-bucket-597703351594/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg
[2024-01-25 22:17:16,164] p16253 {utils.py:33} INFO - downloaded multimodal-bucket-597703351594/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg
[2024-01-25 22:17:16,240] p16253 {utils.py:33} INFO - downloaded multimodal-bucket-597703351594/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg
[2024-01-25 22:17:16,273] p16253 {utils.py:33} INFO - downloaded multimodal-bucket-597703351594/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg
[2024-01-25 22:17:16,357] p16253 {utils.py:33} INFO - downloaded multimodal-bucket-597703351594/multimodal

Convert jpg files into Base64.

In [5]:
def encode_image_to_base64(image_file_path: str) -> str:
    with open(image_file_path, "rb") as image_file:
        b64_image = base64.b64encode(image_file.read()).decode('utf8')
        b64_image_path = os.path.join(g.B64_ENCODED_IMAGES_DIR, f"{Path(image_file_path).stem}.b64")
        with open(b64_image_path, "wb") as b64_image_file:
            b64_image_file.write(bytes(b64_image, 'utf-8'))
    return b64_image_path

In [6]:
os.makedirs(g.B64_ENCODED_IMAGES_DIR, exist_ok=True)
file_list: List = glob.glob(os.path.join(g.IMAGE_DIR, f"*{g.IMAGE_FILE_EXTN}"))
logger.info(f"there are {len(file_list)} files in the {g.IMAGE_DIR} directory for conversion to base64")

# convert each file to base64 and store the base64 in a new file
b64_image_file_list = list(map(encode_image_to_base64, file_list))
logger.info(f"base64 conversion done, there are {len(b64_image_file_list)} base64 encoded files")

[2024-01-25 22:17:18,191] p16253 {4025043965.py:3} INFO - there are 31 files in the img directory for conversion to base64
[2024-01-25 22:17:18,201] p16253 {4025043965.py:7} INFO - base64 conversion done, there are 31 base64 encoded files


## Step 3. Get embeddings for the base64 encoded images

Now we are ready to use Amazon Bedrock via the Amazon Titan Multimodal Embeddings model to convert the base64 version of the images into embeddings. We store these embeddings into a single JSON file which is then uploaded into S3. 

It is important to note that the embeddings corresponding to all the images are stored in a single file so that they can be ingested into the vector database in a single PUT operation (one bulk ingest call is more effecient than one ingest call for each image).

In [7]:
def get_multimodal_embeddings(bedrock: botocore.client, image: str) -> np.ndarray:
    body = json.dumps(dict(inputImage=image))
    try:
        response = bedrock.invoke_model(
            body=body, modelId=g.FMC_MODEL_ID, accept=g.ACCEPT_ENCODING, contentType=g.CONTENT_ENCODING
        )
        response_body = json.loads(response.get("body").read())
        embeddings = np.array([response_body.get("embedding")]).astype(np.float32)
    except Exception as e:
        logger.error(f"exception while image(truncated)={image[:10]}, exception={e}")
        embeddings = None

    return embeddings

In [8]:
embeddings_list = []
bedrock = boto3.client(service_name="bedrock-runtime", region_name=g.AWS_REGION, endpoint_url=g.FMC_URL)
file_list: List = glob.glob(os.path.join(g.B64_ENCODED_IMAGES_DIR, "*.b64"))
logger.info(f"there are {len(file_list)} to convert to embeddings")
for image_file_path in file_list:
    logger.info(f"going to convert {image_file_path} into embeddings")
    
    # read the file, MAX image size supported is 2048 * 2048 pixels
    with open(image_file_path, "rb") as image_file:
        input_image_b64 = image_file.read().decode('utf-8')
    
    # make a call to Bedrock to get the embeddings corresponding to
    # this image's base64 data
    st = time.perf_counter()
    embeddings = get_multimodal_embeddings(bedrock, input_image_b64)
    if embeddings is None:
        logger.error(f"error creating multimodal embeddings for {os.path.basename(image_file_path)}")
        continue
    latency = time.perf_counter() - st
    logger.info(f"successfully converted {image_file_path} to embeddings in {latency:.2f} seconds")
    
    # convert the data we want to ingest for this image into a JSON, this include the metadata as well
    # the metadata can be used later as part of hybrid search from the vector db
    data = {
        "image_path": f"s3://{bucket_name}/{g.BUCKET_IMG_PREFIX}/{Path(image_file_path).stem}{g.IMAGE_FILE_EXTN}",
        "metadata": {
          "slide_filename": g.SLIDE_DECK,
          "model_id": g.FMC_MODEL_ID,
          "slide_description": ""
        },
        "vector_embedding": embeddings[0].tolist()
      }
    
    embeddings_list.append(data)
    logger.info(f"appended json data corresponding to {image_file_path}")

[2024-01-25 22:17:18,226] p16253 {2086549339.py:4} INFO - there are 31 to convert to embeddings
[2024-01-25 22:17:18,227] p16253 {2086549339.py:6} INFO - going to convert img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_2.b64 into embeddings
[2024-01-25 22:17:18,388] p16253 {2086549339.py:20} INFO - successfully converted img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_2.b64 to embeddings in 0.16 seconds
[2024-01-25 22:17:18,389] p16253 {2086549339.py:35} INFO - appended json data corresponding to img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_2.b64
[2024-01-25 22:17:18,390] p16253 {2086549339.py:6} INFO - going to convert img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_27.b64 into embeddings
[2024-01-25 22:17:18,513] p16253 {2086549339.py:20} INFO - successfully converted img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_27.b64 to embeddings in 0.12 seconds
[2024-01-25 22:17:18,515] p16253 {2086549339.py:35} INFO - appende

In [9]:
fpath: str = f"{Path(g.SLIDE_DECK).stem}.json"
json.dump(embeddings_list, codecs.open(fpath, 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)
logger.info(f"saved multimodal embeddings for all images in {fpath}")

[2024-01-25 22:17:23,432] p16253 {1527796339.py:6} INFO - saved multimodal embeddings for all images in CMP301_TrainDeploy_E1_20230607_SPEdited.json


## Step 4. Create the OpenSearch Service Serverless index

**This step is only required until we get support creating an OpenSearch Service Serverless index via AWS CloudFormation**.

Get the name of the OpenSearch Service Serverless collection endpoint and index name from the CloudFormation stack outputs.

In [10]:
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
index_name = outputs['OpenSearchIndexName']
logger.info(f"opensearchhost={host}, index={index_name}")

[2024-01-25 22:17:23,550] p16253 {2210621458.py:4} INFO - opensearchhost=nz2313ydxnjveocqyhvh.us-west-2.aoss.amazonaws.com, index=slides


We use the OpenSearch client to create an index.

In [11]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-01-25 22:17:23,602] p16253 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


The structure of the index is important. Note the following about the index.

1. The index is a (k-NN](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html) index so that we can do a vector similarity search in this index.

1. The vector dimension is 1024 which corresponds to the output dimension of the Amazon Titan Multimodal Embeddings model that we are using.

1. The index uses the [`Hierarchical Navigable Small World (HNSW)`](https://aws.amazon.com/blogs/big-data/choose-the-k-nn-algorithm-for-your-billion-scale-use-case-with-opensearch/) algorithm for similarity search.


In [12]:
index_body = """
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "vector_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {}
        }
      },
      "image_path": {
        "type": "text"
      },
       "metadata": { 
        "properties" :
          {
            "slide_filename" : {
              "type" : "text"
            },
            "model_id" : {
              "type" : "text"
            },
            "slide_description":{
              "type": "text"
            }
          }
      }
    }
  }
}
"""

# We would get an index already exists exception if the index already exists, and that is fine.
index_body = json.loads(index_body)
try:
    response = os_client.indices.create(index_name, body=index_body)
    logger.info(f"response received for the create index -> {response}")
except Exception as e:
    logger.error(f"error in creating index={index_name}, exception={e}")

[2024-01-25 22:17:23,908] p16253 {base.py:281} WARNING - PUT https://nz2313ydxnjveocqyhvh.us-west-2.aoss.amazonaws.com:443/slides [status:400 request:0.297s]
[2024-01-25 22:17:23,910] p16253 {3838889910.py:45} ERROR - error in creating index=slides, exception=RequestError(400, 'resource_already_exists_exception', 'OpenSearch exception [type=resource_already_exists_exception, reason=index [slides/bLyzQo0BPlvDTUGmADs1] already exists]- server : [envoy]')


## Step 5. Upload the embeddings file to S3

Now we are all set for ingesting the embeddings file that contains multimodal embeddings for all the slides in our slide deck into OpenSearch Service Serverless.

We do this by simply uploading the file in the designated S3 bucket (see CloudFormation template [`template.yaml`](../template.yaml)) and that triggers a run of the OpenSearch Ingestion pipeline which ultimately ingests the data into the OpenSearch Service Serverless index.

In [13]:
upload_to_s3(f"{Path(g.SLIDE_DECK).stem}.json", bucket_name, g.BUCKET_EMB_PREFIX)

[2024-01-25 22:17:24,032] p16253 {utils.py:21} INFO - File CMP301_TrainDeploy_E1_20230607_SPEdited.json uploaded to multimodal-bucket-597703351594/multimodal/osi-embeddings-json/CMP301_TrainDeploy_E1_20230607_SPEdited.json.
